In [1]:
# only run this if your have an editable install
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset

fiqa_test = load_dataset("explodinggradients/fiqa", "main", split="test")
fiqa_test

Found cached dataset fiqa (/home/jjmachan/.cache/huggingface/datasets/explodinggradients___fiqa/main/1.0.0/3dc7b639f5b4b16509a3299a2ceb78bf5fe98ee6b5fee25e7d5e4d290c88efb8)


Dataset({
    features: ['question', 'ground_truths'],
    num_rows: 648
})

## k=1, chunk_size=300

We know the performance of the baseline model so lets try and see if we can make improvements to that.

In [3]:
from llama_index import GPTVectorStoreIndex, MockEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext, StorageContext

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
hf_sc = ServiceContext.from_defaults(embed_model=embed_model)

# mock embeddings
embed_model = MockEmbedding(embed_dim=1536)
mock = ServiceContext.from_defaults(embed_model=embed_model)

# openai embeddings
openai_sc = ServiceContext.from_defaults()

In [4]:
# load the index
from llama_index import StorageContext, load_index_from_storage, ServiceContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")

# load index
index = load_index_from_storage(storage_context)

# query with embed_model specified
qe = index.as_query_engine(
    mode="embedding", verbose=True, service_context=openai_sc, use_async=False
)

In [5]:
from llama_index import (
    GPTVectorStoreIndex,
    ResponseSynthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

# configure response synthesizer
response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)

# assemble query engine
qe = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [6]:
def generate_response(row):
    r = qe.query(row["question"])
    row["answer"] = r.response
    row["contexts"] = [sn.node.text for sn in r.source_nodes]

    return row


# generate_response(test_ds[0])

In [9]:
gen_ds = fiqa_test.select(range(30)).map(generate_response)
gen_ds

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [10]:
# evaluate
from ragas.metrics import factuality, answer_relevancy, context_relevancy
from ragas import evaluate

evaluate(gen_ds, metrics=[factuality, answer_relevancy, context_relevancy])

/home/jjmachan/miniconda3/envs/bench/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████| 2/2 [01:18<00:00, 39.17s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.51s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.51s/it]


{'NLI_score': 0.8822222222222222, 'answer_relevancy': 0.8647333333333332, 'context_ relevancy': 0.8236333333333333, 'ragas_score': 0.8561498126750564}

## k=1, chunk=100

In [4]:
# load the index
from llama_index import StorageContext, load_index_from_storage, ServiceContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")

# load index
index = load_index_from_storage(storage_context)

# query with embed_model specified
qe = index.as_query_engine(
    mode="embedding", verbose=True, service_context=openai_sc, use_async=False
)

In [5]:
from llama_index import (
    GPTVectorStoreIndex,
    ResponseSynthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

# configure response synthesizer
response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)

# assemble query engine
qe = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [6]:
def generate_response(row):
    r = qe.query(row["question"])
    row["answer"] = r.response
    row["contexts"] = [sn.node.text for sn in r.source_nodes]

    return row


# generate_response(test_ds[0])

In [7]:
gen_ds = fiqa_test.select(range(30)).map(generate_response)
gen_ds

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [8]:
# evaluate
from ragas.metrics import factuality, answer_relevancy, context_relevancy
from ragas import evaluate

evaluate(gen_ds, metrics=[factuality, answer_relevancy, context_relevancy])

100%|█████████████████████████████████████████████████████████████| 2/2 [00:56<00:00, 28.39s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.04s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.73s/it]


{'ragas_score': 0.8386, 'factuality': 0.8289, 'answer_relevancy': 0.8646, 'context_ relevancy': 0.8236}

In [14]:
# save to fiqa hub
import os

path_to_dataset = "../../../../datasets/fiqa/"

gen_ds.to_csv(os.path.join(path_to_dataset, "baseline_chunk100_k1.csv"))

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

82699

## Cohere Rerank

In [16]:
from llama_index.indices.postprocessor.cohere_rerank import CohereRerank
import os

top_k = 4
cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=top_k)
reranking_qe = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[cohere_rerank],
)

In [17]:
def generate_response(row):
    r = reranking_qe.query(row["question"])
    row["answer"] = r.response
    row["contexts"] = [sn.node.text for sn in r.source_nodes]

    return row


# generate_response(fiqa_test[0])

In [18]:
gen_ds = fiqa_test.select(range(30)).map(generate_response)
gen_ds

Parameter 'function'=<function generate_response at 0x7f832ef4f010> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [24]:
# evaluate
from ragas.metrics import factuality, answer_relevancy, context_relevancy
from ragas import evaluate

evaluate(gen_ds, metrics=[factuality, answer_relevancy, context_relevancy])

100%|█████████████████████████████████████████████████████████████| 2/2 [01:42<00:00, 51.01s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.76s/it]


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

100%|█████████████████████████████████████████████████████████████| 4/4 [01:00<00:00, 15.05s/it]


{'NLI_score': 0.8805555555555556, 'answer_relevancy': 0.8766333333333333, 'context_ relevancy': 0.8163833333333333, 'ragas_score': 0.8568272415550885}

In [3]:
evals["cohere_reranked"] = gen_ds
evals

NameError: name 'gen_ds' is not defined